## Описание и проверка гипотез

### Импорты

In [1]:
import os
import kagglehub
import cv2
import src.image_classification.features as feat
import numpy as np
import scipy.stats as stats
import pandas as pd
import glob

from sklearn.preprocessing import LabelEncoder
from scipy.stats import mannwhitneyu, spearmanr, kruskal, levene, ttest_ind, pearsonr, chi2_contingency, f_oneway

/home/pasha/dump7/image_classification/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Загрузка всех изображений из папки 

In [2]:
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        filepath = os.path.join(folder, filename)
        image = cv2.imread(filepath)
        if image is not None: 
            images.append(image)
    return images

### Основной путь и классы, с которыми будет идти работа

In [3]:
path = kagglehub.dataset_download("sumn2u/garbage-classification-v2")
classes = os.listdir(os.path.join(path, "garbage-dataset"))

### Гипотеза №1 О наличие картона на картинке можно судить по скосу желтого цвета.

В данной гипотезе мы предполагаем, что цветовая характеристика изображения, а именно наличие желтого цвета, может служить индикатором присутствия картона на картинке. Исходя из наблюдений, мы предполагаем, что изображения, содержащие картон, будут иметь более выраженный желтый оттенок по сравнению с изображениями, не содержащими этот материал.


- Способы проверки:

    - **t-критерий Стьюдента (t-test)**: Этот метод будет использован для сравнения средней доли желтого цвета между двумя группами. Мы проверим нулевую гипотезу о том, что средние значения доли желтого цвета в обеих группах не отличаются.

    - **ANOVA (дисперсионный анализ)**: Если у нас есть более двух групп (например, различные виды картона и другие материалы), ANOVA поможет определить, есть ли значимые различия в количестве желтого цвета между всеми группами.

    - **Критерий Манна-Уитни (U-тест)**: Непараметрический тест для сравнения распределений двух групп (в данном случае картон и другие классы).

In [4]:
cardboard_path = os.path.join(path + "/garbage-dataset", "cardboard")
other_paths = [os.path.join(path + "/garbage-dataset", cls) for cls in classes if cls != "cardboard"]

yellow_feature = feat.AmountOfYellow()

cardboard_images = load_images_from_folder(cardboard_path)
cardboard_yellow_amounts = [yellow_feature(img) for img in cardboard_images]

other_yellow_amounts = []
for other_path in other_paths:
    other_images = load_images_from_folder(other_path)
    other_yellow_amounts.extend([yellow_feature(img) for img in other_images])

# Проверка гипотез
# 1. t-критерий Стьюдента
t_stat, t_pvalue = ttest_ind(cardboard_yellow_amounts, other_yellow_amounts, equal_var=False)

# 2. ANOVA
anova_stat, anova_pvalue = f_oneway(cardboard_yellow_amounts, other_yellow_amounts)

# 3. Критерий Манна-Уитни
mann_stat, mann_pvalue = mannwhitneyu(cardboard_yellow_amounts, other_yellow_amounts, alternative='two-sided')

results = {
    "t-test": {"statistic": t_stat, "p-value": t_pvalue},
    "ANOVA": {"statistic": anova_stat, "p-value": anova_pvalue},
    "Mann-Whitney": {"statistic": mann_stat, "p-value": mann_pvalue},
}

for test, result in results.items():
    print(f"{test}: Statistic = {result['statistic']}, p-value = {result['p-value']}")

alpha = 0.05
hypothesis_results = {test: result['p-value'] < alpha for test, result in results.items()}

if all(hypothesis_results.values()):
    print("Итог: гипотеза подтверждена: наличие картона связано c желтым цветом.")
else:
    print("Итог: гипотеза не подтверждена: наличие картона не связано c желтым цветом.")

t-test: Statistic = 3.683066343249196, p-value = 0.00023621963453461276
ANOVA: Statistic = 16.865194578791744, p-value = 4.0292196417557166e-05
Mann-Whitney: Statistic = 22080315.5, p-value = 1.0126318223771204e-146
Итог: гипотеза подтверждена: наличие картона связано c желтым цветом.


### Гипотеза №2 О наличие металла на картинке можно судить по скосу серого (серебряного) цвета.

В данной гипотезе мы предполагаем, что цветовая характеристика изображения, а именно наличие серого или серебряного цвета, может служить индикатором присутствия металла на картинке. Мы предполагаем, что изображения, содержащие металл, будут иметь более выраженный серый оттенок по сравнению с изображениями других классов.

- Способы проверки:

    - **t-критерий Стьюдента (t-test)**: Этот метод будет использован для сравнения средней доли серебряного цвета между двумя группами. Мы проверим нулевую гипотезу о том, что средние значения доли серебряного цвета в обеих группах не отличаются.

    - **ANOVA (дисперсионный анализ)**: Если у нас есть более двух групп (например, различные виды металлов и другие материалы), ANOVA поможет определить, есть ли значимые различия в количестве серебряного цвета между всеми группами. 

    - **Критерий Манна-Уитни (U-тест)**: Непараметрический тест для сравнения распределений двух групп (в данном случае металл и другие классы).

In [5]:
metal_path = os.path.join(path + "/garbage-dataset", "metal")
other_paths = [os.path.join(path + "/garbage-dataset", cls) for cls in classes if cls != "metal"]

silver_feature = feat.AmountOfSilver()

metal_images = load_images_from_folder(metal_path)
metal_silver_amounts = [silver_feature(img) for img in metal_images]

other_silver_amounts = []
for other_path in other_paths:
    other_images = load_images_from_folder(other_path)
    other_silver_amounts.extend([silver_feature(img) for img in other_images])

# Проверка гипотез
# 1. t-критерий Стьюдента
t_stat, t_pvalue = ttest_ind(metal_silver_amounts, other_silver_amounts, equal_var=False)

# 2. ANOVA
anova_stat, anova_pvalue = f_oneway(metal_silver_amounts, other_silver_amounts)

# 3. Критерий Манна-Уитни
mann_stat, mann_pvalue = mannwhitneyu(metal_silver_amounts, other_silver_amounts, alternative='two-sided')

results = {
    "t-test": {"statistic": t_stat, "p-value": t_pvalue},
    "ANOVA": {"statistic": anova_stat, "p-value": anova_pvalue},
    "Mann-Whitney": {"statistic": mann_stat, "p-value": mann_pvalue},
}

for test, result in results.items():
    print(f"{test}: Statistic = {result['statistic']}, p-value = {result['p-value']}")

alpha = 0.05
hypothesis_results = {test: result['p-value'] < alpha for test, result in results.items()}

if all(hypothesis_results.values()):
    print("Итог: гипотеза подтверждена: наличие металла связано c серебряным цветом.")
else:
    print("Итог: гипотеза не подтверждена: наличие металла не связано c серебряным цветом.")

t-test: Statistic = -2.284424004913432, p-value = 0.022401779559765453
ANOVA: Statistic = 0.5733189930562239, p-value = 0.448952071385762
Mann-Whitney: Statistic = 10996543.5, p-value = 5.262970510833023e-16
Итог: гипотеза не подтверждена: наличие металла не связано c серебряным цветом.


### Гипотеза №3 На картинках с картоном присутствует множество параллельных линий.

На картинках с картоном присутствует множество параллельных линий, что может быть связано с характерными текстурами и структурой материала. Данная гипотеза предполагает, что изображения с картоном содержат больше параллельных линий по сравнению с изображениями других классов.

- Способы проверки:

    - **t-критерий Стьюдента (t-test)**: тот метод позволяет сравнить среднюю долю параллельных линий в картинках с картоном и других классах. Мы рассчитываем t-статистику и p-значение, чтобы определить, есть ли статистически значимые различия между двумя группами.

    - **ANOVA (дисперсионный анализ)**: Данный подход позволяет проверить, существуют ли значимые различия между несколькими группами (в данном случае между картоном и другими классами) по количеству параллельных линий. ANOVA помогает определить, влияет ли класс изображения на количество параллельных линий.

    - **Критерий хи-квадрат**: Этот тест позволяет проверить гипотезу о различии долей изображений с высокими количествами параллельных линий между картоном и другими классами. Мы анализируем, сколько изображений из каждой группы попадает в категорию с высоким количеством параллельных линий и сравниваем эти данные.

In [6]:
cardboard_path = os.path.join(path + "/garbage-dataset", "cardboard")
other_paths = [os.path.join(path + "/garbage-dataset", cls) for cls in classes if cls != "cardboard"]

cardboard_images = load_images_from_folder(cardboard_path)

parallel_lines_feature_extractor = feat.AmountOfParallelLines()

cardboard_features = [parallel_lines_feature_extractor(img) for img in cardboard_images]

other_features = []
for other_path in other_paths:
    other_images = load_images_from_folder(other_path)
    other_features.extend([parallel_lines_feature_extractor(img) for img in other_images])

# Hypothesis testing
# 1. Student's t-test
t_stat, t_pvalue = ttest_ind(cardboard_features, other_features, equal_var=False)

# 2. ANOVA
anova_stat, anova_pvalue = f_oneway(cardboard_features, other_features)

# 3. Chi-square test
threshold = np.percentile(cardboard_features, 75) 
cardboard_high = sum(x >= threshold for x in cardboard_features)
cardboard_low = len(cardboard_features) - cardboard_high
other_high = sum(x >= threshold for x in other_features)
other_low = len(other_features) - other_high
chi2_stat, chi2_pvalue, _, _ = chi2_contingency([[cardboard_high, cardboard_low], [other_high, other_low]])

results = {
    "t-test": {"statistic": t_stat, "p-value": t_pvalue},
    "ANOVA": {"statistic": anova_stat, "p-value": anova_pvalue},
    "Chi-square": {"statistic": chi2_stat, "p-value": chi2_pvalue},
}

for test, result in results.items():
    print(f"{test}: Statistic = {result['statistic']}, p-value = {result['p-value']}")

alpha = 0.05
hypothesis_results = {test: result['p-value'] < alpha for test, result in results.items()}

if all(hypothesis_results.values()):
    print("Итог: гипотеза подтверждена: изображения c картоном имеют больше параллельных линий.")
else:
    print("Итог: гипотеза не подтверждена: изображения c картоном не имеют значительно больше параллельных линий.")

t-test: Statistic = -11.379057124078555, p-value = 1.9776123451846287e-29
ANOVA: Statistic = 52.405427227996164, p-value = 4.680228101392663e-13
Chi-square: Statistic = 19.34257889433447, p-value = 1.0924297310604447e-05
Итог: гипотеза подтверждена: изображения c картоном имеют больше параллельных линий.


### Гипотеза №4 На изображениях со стеклом и на изображениях с металлом присутствуют цилиндрические формы, окружности.

Цилиндрические формы и окружности являются характерными геометрическими элементами, которые могут встречаться на изображениях, содержащих стекло и металл.

- Способы проверки:
    - **ANOVA (дисперсионный анализ)**: Этот метод позволит нам проверить наличие значимых различий в количестве цилиндрических форм между группами (стекло, металл и другие классы). ANOVA поможет определить, существуют ли статистически значимые различия между средними значениями количества цилиндров в различных классах.

    - **Корреляция Спирмена**: С помощью этого теста мы проанализируем взаимосвязь между классами изображений и количеством цилиндрических форм, что позволит оценить, насколько сильно класс материала влияет на количество цилиндров.

In [5]:
base_path = path + "/garbage-dataset/"

def process_images(class_name):
    class_path = os.path.join(base_path, class_name)
    cylinder_counts = []

    for filename in os.listdir(class_path):
        image_path = os.path.join(class_path, filename)
        image = cv2.imread(image_path)
        if image is not None:
            cylinder_counts.append(feat.AmountOfCylinders()(image))

    return cylinder_counts

cylinder_data = {}
for cls in classes:
    cylinder_data[cls] = process_images(cls)

# 1. ANOVA для сравнения всех классов
anova_data = [cylinder_data[cls] for cls in classes]
anova_stat, anova_p = f_oneway(*anova_data)

# 2. Корреляция Спирмена между классами и количеством цилиндров
class_labels = []
cylinder_counts = []
for idx, cls in enumerate(classes):
    class_labels.extend([idx] * len(cylinder_data[cls]))
    cylinder_counts.extend(cylinder_data[cls])

spearman_corr, spearman_p = spearmanr(class_labels, cylinder_counts)


print("Результаты тестов:")
print(f"ANOVA: Statistic = {anova_stat}, p-value = {anova_p}")
print(f"Pearson Correlation: Correlation = {spearman_corr}, p-value = {spearman_p}")


if anova_p < 0.05 and spearman_p < 0.05 :
    print("Итог: гипотеза подтверждена: цилиндры чаще встречаются на изображениях стекла и металла.")
else:
    print("Итог: гипотеза не подтверждена: не подтверждена.")


Результаты тестов:
ANOVA: Statistic = 20.58609333952457, p-value = 5.975820361191762e-35
Pearson Correlation: Correlation = 0.18124483155277124, p-value = 1.498345430277706e-145
Итог: гипотеза подтверждена: цилиндры чаще встречаются на изображениях стекла и металла.


### Гипотеза №5 Металлические поверхности часто имеют высокую отражающую способность. 

Металлические поверхности, как правило, обладают высокой отражающей способностью, что делает их уникальными в сравнении с другими материалами. Высокая отражающая способность металла может влиять на визуальное восприятие объектов. В данном эксперименте мы стремимся проверить, действительно ли металлические поверхности обладают более высоким уровнем отражений по сравнению с другими классами материалов.

- Способы проверки:

    - **t-критерий Стьюдента (t-test):**  Этот метод позволяет сравнить среднюю отражающую способность изображений металла и других классов. Мы будем использовать его для определения, существуют ли статистически значимые различия между этими двумя группами.

    - **Корреляция Пирсона:** С помощью этого теста мы проверим наличие линейной связи между количеством отражений и принадлежностью к классу "металл". Это поможет нам понять, насколько сильно отражающая способность связана с материалом.

    - **Критерий хи-квадрат:** Этот тест будет использован для проверки отличий доли изображений с высоким уровнем отражений между металлом и другими классами. Мы разделим изображения на две группы: с высоким уровнем отражений и с низким, и проверим, есть ли статистически значимые различия между ними.

In [7]:
metal_path = os.path.join(path + "/garbage-dataset", "metal")
other_paths = [os.path.join(path + "/garbage-dataset", cls) for cls in classes if cls != "metal"]

reflection_calculator = feat.AmountOfReflections()

metal_images = load_images_from_folder(metal_path)
metal_reflections = [reflection_calculator(img) for img in metal_images]

other_reflections = []
for other_path in other_paths:
    other_images = load_images_from_folder(other_path)
    other_reflections.extend([reflection_calculator(img) for img in other_images])

# 1. t-критерий Стьюдента
t_stat, t_pvalue = ttest_ind(metal_reflections, other_reflections, equal_var=False)

# 2. Корреляция Пирсона
le = LabelEncoder()
all_reflections = metal_reflections + other_reflections
all_labels = ['metal'] * len(metal_reflections) + ['other'] * len(other_reflections)
numeric_labels = le.fit_transform(all_labels)

correlation, p_value_corr = pearsonr(all_reflections, numeric_labels)

# 3. Тест хи-квадрат
reflection_binary = [1 if reflection >= 100 else 0 for reflection in all_reflections]
contingency_table = pd.crosstab(reflection_binary, numeric_labels)
chi2_stat, p_value_chi2, _, _ = chi2_contingency(contingency_table)

results = {
    "t-test": {"statistic": t_stat, "p-value": t_pvalue},
    "Pearson Correlation": {"statistic": correlation, "p-value": p_value_corr},
    "Chi-square": {"statistic": chi2_stat, "p-value": p_value_chi2},
}

for test, result in results.items():
    print(f"{test}: Statistic = {result['statistic']}, p-value = {result['p-value']}")

alpha = 0.05
hypothesis_results = {test: result['p-value'] < alpha for test, result in results.items()}

if all(hypothesis_results.values()):
    print("Итог: гипотеза подтверждена: наличие металла связано с высоким уровнем отражений.")
else:
    print("Итог: гипотеза не подтверждена: наличие металла не связано с высоким уровнем отражений.")

t-test: Statistic = -2.5612454595984153, p-value = 0.010464870898548239
Pearson Correlation: Statistic = 0.005891723334828736, p-value = 0.407558471373331
Chi-square: Statistic = 35.91242941824723, p-value = 2.0638790312840235e-09
Итог: гипотеза не подтверждена: наличие металла не связано с высоким уровнем отражений.


### Гипотеза №6: Изображения с пластиком и стеклом часто имеют больше прозрачных областей.

В данной гипотезе мы предполагаем, что изображения, содержащие пластик и стекло, будут иметь больше прозрачных областей по сравнению с изображениями других классов. Это может быть связано с природой этих материалов, которые часто обладают высокой степенью прозрачности и могут создавать визуально заметные прозрачные участки на изображениях.

- Способы проверки:

    - **Тест Манна-Уитни:**  Этот тест будет использоваться для сравнения распределений количества прозрачных областей на изображениях, содержащих пластик и стекло, с распределением на изображениях других классов.

    - **Коэффициент корреляции Спирмена:** Мы применим этот метод для оценки монотонной зависимости между количеством прозрачных областей и классом изображения. Это позволит нам понять, насколько сильно количество прозрачных областей связано с принадлежностью изображения к классу "пластик" или "стекло" по сравнению с другими классами.

    - **Тест Крускала-Уоллиса:** Этот тест будет использоваться для сравнения медианных значений количества прозрачных областей между тремя группами: пластик, стекло и другие классы. 

In [15]:
plastic_glass_classes = ['plastic', 'glass']
transparency_data = {'plastic': [], 'glass': [], 'other': []}

for class_name in classes:
    class_path = os.path.join(path + "/garbage-dataset/", class_name)
    image_paths = glob.glob(os.path.join(class_path, "*.jpg")) 

    for image_path in image_paths:
        image = cv2.imread(image_path)
        transparency = feat.AmountOfTransparency()(image)

        if class_name in plastic_glass_classes:
            transparency_data['plastic'].append(transparency)
            transparency_data['glass'].append(transparency)
        else:
            transparency_data['other'].append(transparency)

# 1. Тест Манна-Уитни 
u_statistic, p_value_mw = mannwhitneyu(transparency_data['plastic'] + transparency_data['glass'], transparency_data['other'])

# 2. Коэффициент корреляции Спирмена
all_transparency = transparency_data['plastic'] + transparency_data['glass'] + transparency_data['other']
labels = ['plastic_glass'] * (len(transparency_data['plastic']) + len(transparency_data['glass'])) + ['other'] * len(transparency_data['other'])

spearman_corr, p_value_spearman = spearmanr(all_transparency, [1 if label == 'plastic_glass' else 0 for label in labels])

# 3. Тест Крускала-Уоллиса
kruskal_stat, p_value_kruskal = kruskal(transparency_data['plastic'], transparency_data['glass'], transparency_data['other'])

print(f"Mann-Whitney U test: Statistic = {u_statistic}, p-value = {p_value_mw}")
print(f"Spearman correlation: Correlation = {spearman_corr}, p-value = {p_value_spearman}")
print(f"Kruskal-Wallis test: Statistic = {kruskal_stat}, p-value = {p_value_kruskal}")

alpha = 0.05
if p_value_mw < alpha and p_value_spearman < alpha and p_value_kruskal < alpha:
    print("Итог: гипотеза подтверждена: пластик и стекло имеют больше прозрачных областей.")
else:
    print("Итог: гипотеза не подтверждена: нет статистически значимых различий между классами.")

Mann-Whitney U test: Statistic = 63373648.0, p-value = 2.0179651032763478e-82
Spearman correlation: Correlation = -0.12222108103161104, p-value = 5.038550422304453e-83
Kruskal-Wallis test: Statistic = 369.849759985094, p-value = 4.876913498602153e-81
Итог: гипотеза подтверждена: пластик и стекло имеют больше прозрачных областей.


### Гипотеза №7: Металлы часто имеют гладкую текстуру по сравнению с другими материалами.

В данной гипотезе мы выдвигаем предположение, что изображения, содержащие металлические материалы, будут демонстрировать более гладкую текстуру по сравнению с изображениями других классов, таких как пластик, стекло и различные виды отходов. Это связано с физическими свойствами металлов, которые часто обладают высокой отражательной способностью и гладкой поверхностью.

- Способы проверки:

    - **Тест Манна-Уитни**: Этот тест будет применяться для сравнения распределений количества отражений на изображениях, содержащих металл, с распределением на изображениях других классов. 
    
    - **Коэффициент корреляции Спирмена**: Мы используем этот метод для оценки монотонной зависимости между количеством отражений и классом изображения. Это поможет определить, насколько сильно количество отражений связано с принадлежностью изображения к классу "металл" по сравнению с другими классами.

    - **Дисперсионный анализ (ANOVA)**: Этот метод будет использоваться для сравнения средних значений количества отражений между группами: металл и другие материалы. ANOVA позволяет оценить, есть ли статистически значимые различия в средних значениях между несколькими группами.

In [17]:
metal_path = os.path.join(path + "/garbage-dataset", "metal")
other_classes = [cl for cl in classes if cl != "metal"]

def collect_reflections(class_dirs):
    reflections = []
    for class_dir in class_dirs:
        for img_file in os.listdir(class_dir):
            img_path = os.path.join(class_dir, img_file)
            image = cv2.imread(img_path)
            if image is not None:
                reflections.append(feat.AmountOfReflections()(image))
    return reflections

metal_reflections = collect_reflections([metal_path])
other_reflections = collect_reflections([os.path.join(path + "/garbage-dataset", cls) for cls in other_classes])

# 1. Тест Манна-Уитни 
u_stat, p_value_mwu = mannwhitneyu(metal_reflections, other_reflections, alternative='two-sided')

# 2. Корреляция Спирмена 
all_reflections = metal_reflections + other_reflections
class_labels = ['metal'] * len(metal_reflections) + ['other'] * len(other_reflections)

spearman_corr, p_value_spearman = spearmanr(all_reflections, [0]*len(metal_reflections) + [1]*len(other_reflections))

# 3. Дисперсионный анализ (ANOVA) 
all_reflections_per_class = [
    metal_reflections,
    other_reflections
] 

anova_stat, p_value_anova = f_oneway(*all_reflections_per_class)

print(f'Mann-Whitney U Test p-value: {p_value_mwu}')
print(f'Spearman correlation p-value: {p_value_spearman}')
print(f'ANOVA p-value: {p_value_anova}')

if p_value_mwu < 0.05 and p_value_spearman < 0.05 and p_value_anova < 0.05:
    print("Итог: гипотеза подтверждена: Металлы имеют значимые отличия в количестве отражений по сравнению с другими материалами.")
else:
    print("Итог: гипотеза не подтверждена: Нет значимых отличий между металлом и другими материалами.")


Mann-Whitney U Test p-value: 8.969264820020402e-20
Spearman correlation p-value: 8.239260503906844e-20
ANOVA p-value: 0.4075584713849223
Гипотеза не подтверждена: Нет значимых отличий между металлом и другими материалами.


### Гипотеза №8: Металл и пластик часто имеют яркие участки, особенно под прямым освещением.

В данной гипотезе предполагается, что изображения, содержащие металлические и пластиковые материалы, будут демонстрировать более яркие участки, особенно при прямом освещении, по сравнению с изображениями других материалов.

- Способы проверки:

    - **Тест Манна-Уитни**: тот тест будет использоваться для сравнения распределений количества ярких пикселей (отражений) на изображениях с металлом и пластиком по сравнению с изображениями других классов.

    - **Корреляция Пирсона**: Этот метод позволит оценить линейную зависимость между количеством ярких пикселей и классом материала (металл и пластик против других).

    - **Тест Левене**: Этот тест будет применяться для проверки равенства дисперсий яркости между изображениями с металлом и пластиком и изображениями с другими материалами. Он поможет определить, есть ли статистически значимые различия в вариации яркости между двумя группами.

In [23]:
shininess_data = {cls: [] for cls in classes}

for cls in classes:
    class_path = os.path.join(path + "/garbage-dataset/", cls)
    for image_name in os.listdir(class_path):
        if image_name.endswith('.jpg') or image_name.endswith('.png'):
            img_path = os.path.join(class_path, image_name)
            image = cv2.imread(img_path)
            shininess = feat.AmountOfTextureShininess()(image)
            shininess_data[cls].append(shininess)

metal_plastic_shininess = np.array(shininess_data['metal'] + shininess_data['plastic'])
other_shininess = []

for cls in classes:
    if cls not in ['metal', 'plastic']:
        other_shininess.extend(shininess_data[cls])

other_shininess = np.array(other_shininess)

# 1. Тест Манна-Уитни 
u_statistic, p_value_metal_plastic_other = stats.mannwhitneyu(metal_plastic_shininess, other_shininess)

# 2. Корреляция Пирсона между яркостью и классом 
shininess_values = np.concatenate([metal_plastic_shininess, other_shininess])
labels = np.concatenate([np.ones(len(metal_plastic_shininess)), np.zeros(len(other_shininess))])

correlation, p_val_pierson = stats.pearsonr(shininess_values, labels)

# 3. Тест Левене на равенство дисперсий
levene_statistic, p_value_levene = stats.levene(metal_plastic_shininess, other_shininess)

print("Mann-Whitney U Test: p-value =", p_value_metal_plastic_other)
print("Pearson correlation = ", correlation)
print("Levene's test: p-value =", p_value_levene)

if p_value_metal_plastic_other < 0.05 and p_val_pierson < 0.05 and p_value_levene < 0.05:
    print("\nИтог: гипотеза подтверждена: металл и пластик имеют яркие участки")
else:
    print("\nИтог: гипотеза не подтверждена: что металл и пластик не имеют яркие участки")


Mann-Whitney U Test: p-value = 0.24426556591075066
Pearson correlation =  -0.015498280007591004
Levene's test: p-value = 0.0123128462684152

Итог: гипотеза не подтверждена: что металл и пластик не имеют яркие участки


### Гипотеза №9: У металла анизотропия текстуры (ориентированность линий) может быть выше из-за шлифовки или полос.

В данной гипотезе предполагается, что изображения, содержащие металлические материалы, будут демонстрировать более высокую анизотропию текстуры по сравнению с изображениями других классов (например, стекло, бумага и другие). Это может быть связано с процессами обработки металла, такими как шлифовка, которые создают ориентированные линии на поверхности.

- Способы проверки:

    - **Тест Манна-Уитни**: Этот тест будет использоваться для сравнения распределений анизотропии текстуры между изображениями с металлом и изображениями с другими классами.

    - **Корреляция Спирмена**: Этот метод позволит оценить зависимость средней анизотропии текстуры от классов материала.

    - **Тест Краскела-Уоллиса**:  Этот тест будет применяться для проверки различий в распределении анизотропии текстуры между всеми классами материалов.

In [24]:
anisotropy_data = {}

for class_name in classes:
    class_path = os.path.join(path + "/garbage-dataset/", class_name)
    anisotropy_data[class_name] = []

    for img_file in os.listdir(class_path):
        img_path = os.path.join(class_path, img_file)
        image = cv2.imread(img_path)
        if image is not None:
            anisotropy = feat.AmountOfSurfaceAnisotropy()(image)
            anisotropy_data[class_name].append(anisotropy)

metal_anisotropy = anisotropy_data['metal']
non_metal_anisotropy = [value for class_name, values in anisotropy_data.items() if class_name != 'metal' for value in values]

# 1. Тест Манна-Уитни
mannwhitney_result = mannwhitneyu(metal_anisotropy, non_metal_anisotropy, alternative='greater')

# 2. Тест Краскела-Уоллиса
kruskal_result = kruskal(metal_anisotropy, *[anisotropy_data[class_name] for class_name in classes if class_name != 'metal'])

# 3. Корреляция Спирмена между классами
mean_anisotropies = [np.mean(anisotropy_data[class_name]) for class_name in classes]
spearman_corr, spearman_pvalue = spearmanr(range(len(classes)), mean_anisotropies)

print("Результаты тестов:")
print(f"Mann-Whitney U Test: p-value = {mannwhitney_result.pvalue}")
print(f"Kruskal-Wallis test: p-value = {kruskal_result.pvalue}")
print(f"Spearman correlation: Correlation = {spearman_corr}, p-value = {spearman_pvalue}")

if mannwhitney_result.pvalue < 0.05 and kruskal_result.pvalue < 0.05 and spearman_pvalue < 0.05:
    print("Итог: гипотеза подтверждена: текстурная анизотропия металла выше.")
else:
    print("Итог: гипотеза не подтверждена: нет статистически значимых доказательств.")


Результаты тестов:
1. Тест Манна-Уитни U: p-value = 0.07172223142141215
2. Тест Краскела-Уоллиса: p-value = 0.0
3. Корреляция Спирмена: корреляция = -0.5636363636363636, p-value = 0.08972402831709125
Гипотеза не подтверждена: нет статистически значимых доказательств.


### Гипотеза №10: Металл и стекло имеют более постоянные размеры и пропорции на изображениях. 

В данной гипотезе мы предполагаем, что объекты, изготовленные из металла и стекла, обладают более стабильными аспектными соотношениями по сравнению с другими классами объектов. Это может быть связано с их физическими свойствами, которые обеспечивают меньшую вариативность в размерах и формах при производстве и обработке.

- Способы проверки:

    - **Критерий Манна-Уитни:** Мы будем использовать критерий Манна-Уитни для анализа различий в средних значениях аспектных соотношений между группой "металл и стекло" и всеми остальными классами.

    - **Критерия Левена:** Для проверки однородности дисперсий аспектных соотношений между группой "металл и стекло" и другими классами будет применен критерий Левена. 

    - **критерия Спирмена:** Проведем анализ корреляции между классами объектов (металл/стекло против остальных) и аспектными соотношениями с помощью коэффициента корреляции Спирмена. Этот метод позволит оценить, существует ли статистически значимая связь между типом материала и стабильностью его размеров.


In [10]:
def get_aspect_ratios(class_name):
    class_path = os.path.join(base_path, class_name)
    aspect_ratios = []
    aspect_ratio_calculator = feat.AmountOfAspectRatio()

    for image_name in os.listdir(class_path):
        image_path = os.path.join(class_path, image_name)
        image = cv2.imread(image_path) 
        if image is not None: 
            ratio = aspect_ratio_calculator(image) 
            if ratio is not None:
                aspect_ratios.append(ratio)

    return aspect_ratios

metal_ratios = get_aspect_ratios('metal')
glass_ratios = get_aspect_ratios('glass')
other_ratios = []

for cls in classes:
    if cls not in ['metal', 'glass']:
        other_ratios.extend(get_aspect_ratios(cls))

glass_metal_ratios = metal_ratios + glass_ratios

# 1. Тест Манна-Уитни
mann_whitney_result = mannwhitneyu(glass_metal_ratios, other_ratios, alternative='two-sided')

# 2. Тест Левена
levene_result = levene(glass_metal_ratios, other_ratios)

# 3. Корреляция Спирмена
labels = [1] * len(glass_metal_ratios) + [0] * len(other_ratios) 
all_ratios = glass_metal_ratios + other_ratios
spearman_result = spearmanr(labels, all_ratios)


print(f"Mann-Whitney U test: p-value = {mann_whitney_result.pvalue}")
print(f"Levene's test: p-value = {levene_result.pvalue}")
print(f"Spearman correlation: Correlation = {spearman_result.correlation}, p-value = {spearman_result.pvalue}")

alpha = 0.05
if (mann_whitney_result.pvalue < alpha) and (levene_result.pvalue < alpha) and (spearman_result.pvalue < alpha):
    print("Итог: гипотеза подтверждена: металл и стекло имеют более постоянные размеры и пропорции.")
else:
    print("Итог: гипотеза не подтверждена: нет значимых различий в пропорциях.")

Mann-Whitney U test: p-value = 0.21303798516232975
Levene's test: p-value = 0.05283491837140859
Spearman correlation: Correlation = -0.008858382256446987, p-value = 0.21304580872135098
Итог: гипотеза не подтверждена: нет значимых различий в пропорциях.


### Гипотеза №11 О наличие бумаги на картинке можно судить по скосу белого цвета.

В этой гипотезе предполагается, что изображения, содержащие бумагу, будут иметь более высокую долю белого цвета по сравнению с изображениями других классов.

- Способы проверки:

    - **t-критерий Стьюдента (t-test)**: Сравнение средней доли белого цвета в картинках с бумагой и другими классами.

    - **ANOVA (дисперсионный анализ)**: Проверка, есть ли значимые различия между группами (бумага и другие классы) по количеству белого цвета.

In [8]:
paper_path = os.path.join(path + "/garbage-dataset", "paper")
other_paths = [os.path.join(path + "/garbage-dataset", cls) for cls in classes if cls != "paper"]

paper_images = load_images_from_folder(paper_path)
paper_white_amounts = [feat.AmountOfWhiteness()(img) for img in paper_images]

other_white_amounts = []
for other_path in other_paths:
    other_images = load_images_from_folder(other_path)
    other_white_amounts.extend([feat.AmountOfWhiteness()(img) for img in other_images])

# Проверка гипотез
# 1. t-критерий Стьюдента
t_stat, t_pvalue = ttest_ind(paper_white_amounts, other_white_amounts, equal_var=False)

# 2. ANOVA
anova_stat, anova_pvalue = f_oneway(paper_white_amounts, other_white_amounts)

results = {
    "t-test": {"statistic": t_stat, "p-value": t_pvalue},
    "ANOVA": {"statistic": anova_stat, "p-value": anova_pvalue},
}

for test, result in results.items():
    print(f"{test}: Statistic = {result['statistic']}, p-value = {result['p-value']}")

alpha = 0.05
hypothesis_results = {test: result['p-value'] < alpha for test, result in results.items()}

if all(hypothesis_results.values()):
    print("Итог: гипотеза подтверждена: наличие бумаги связано c белым цветом.")
else:
    print("Итог: гипотеза не подтверждена: наличие бумаги не связано c белым цветом.")

t-test: Statistic = -3.6453611581374994, p-value = 0.00027298340264948675
ANOVA: Statistic = 8.183087235905052, p-value = 0.004232707066685615
Итог: гипотеза подтверждена: наличие бумаги связано c белым цветом.


### Гипотеза №12 Одежду можно отличить от других классов по наличию волнистых краёв или складок, которые часто встречаются на ткани.

В данной гипотезе предполагается, что изображения одежды будут иметь большее количество складок и волнистых краёв по сравнению с изображениями других классов.

- Способы проверки:

    - **t-критерий Стьюдента (t-test)**: Сравнение среднего количества складок в картинках с одеждой и другими классами.

    - **ANOVA (дисперсионный анализ)**: Проверка, есть ли значимые различия между группами (одежда и другие классы) по количеству складок.

    - **Критерий Манна-Уитни (U-тест)**: тест для сравнения распределений двух групп (в данном случае одежда и другие классы).

In [34]:
clothes_path = path + "/garbage-dataset/clothes"
other_classes = [cl for cl in classes if cl != "clothes"]
data = {"clothes": [], "others": []}

for img_name in os.listdir(clothes_path):
    img_path = os.path.join(clothes_path, img_name)
    img = cv2.imread(img_path)
    if img is not None:
        curvature = feat.AmountOfLineCurvature()(img)
        data["clothes"].append(curvature)

for cls in other_classes:
    class_path = path + "/garbage-dataset/" + cls
    for img_name in os.listdir(class_path):
        img_path = os.path.join(class_path, img_name)
        img = cv2.imread(img_path)
        if img is not None:
            curvature = feat.AmountOfLineCurvature()(img)
            data["others"].append(curvature)

clothes_curvature = np.array(data["clothes"])
others_curvature = np.array(data["others"])

# t-тест Стьюдента
t_stat, t_p_value = ttest_ind(clothes_curvature, others_curvature, equal_var=False)

# ANOVA
a_stat, a_p_value = f_oneway(clothes_curvature, others_curvature)

# U-тест Манна-Уитни
u_stat, u_p_value = mannwhitneyu(clothes_curvature, others_curvature)

if t_p_value < 0.05 and a_p_value < 0.05 and u_p_value < 0.05:
    print("Итог: гипотеза подтверждена: одежду можно отличить от других классов по наличию волнистых краёв или складок.")
else:
    print("Итог: гипотеза не подтверждена: различия между классами недостаточно статистически значимы.")

Итог: гипотеза не подтверждена: различия между классами недостаточно статистически значимы.


### Выводы
#### Были подтверждены следующие гипотезы:

- Гипотеза №1 О наличие картона на картинке можно судить по скосу желтого цвета.
- Гипотеза №3 На картинках с картоном присутствует множество параллельных линий.
- Гипотеза №4 На изображениях со стеклом и на изображениях с металлом присутствуют цилиндрические формы, окружности.
- Гипотеза №6: Изображения с пластиком и стеклом часто имеют больше прозрачных областей.
- Гипотеза №11 О наличие бумаги на картинке можно судить по скосу белого цвета.

#### Были опровергнуты следующие гипотезы:

- Гипотеза №2 О наличие металла на картинке можно судить по скосу серого (серебряного) цвета.
- Гипотеза №5 Металлические поверхности часто имеют высокую отражающую способность. 
- Гипотеза №7: Металлы часто имеют гладкую текстуру по сравнению с другими материалами.
- Гипотеза №8: Металл и пластик часто имеют яркие участки, особенно под прямым освещением.
- Гипотеза №9: У металла анизотропия текстуры (ориентированность линий) может быть выше из-за шлифовки или полос.
- Гипотеза №10: Металл и стекло имеют более постоянные размеры и пропорции на изображениях. 
- Гипотеза №12 Одежду можно отличить от других классов по наличию волнистых краёв или складок, которые часто встречаются на ткани.